#This is a personal project to evaluate net money flows into Brazilian funds from 2021 to 2023 using CVM (Brazilian SEC) raw data. 
#Please note that it was a "learn as you code" personal project using AI helpers and YouTube Tutorials. Comments are welcome!

In [ ]:
import pandas as pd
import requests
import os
import zipfile
import io
import matplotlib.pyplot as plt

In [ ]:
#Download the files:
# Specify the base URL
base_url = "https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{:02d}{:02d}.zip"

# Specify the range of years you want to download
start_year = 2021
end_year = 2023
# Specify the range of years you want to download
start_month = 1
end_month = 12

# Specify the directory to save the downloaded files

download_directory = "Fundos_555"

# Create the download directory if it doesn't exist
os.makedirs(download_directory, exist_ok=True)

# Loop through the years and download the files
for year in range(start_year, end_year + 1):
    for month in range(start_month, end_month + 1):
        # Create the URL for each month
        url = base_url.format(year, month)
        filename = f"inf_diario_fi_{year}{month}.zip"
        filepath = os.path.join(download_directory, filename)

        # Download the file
        response = requests.get(url)
        with open(filepath, 'wb') as file:
            file.write(response.content)

        print(f"Downloaded {filename}")

print("Download complete!")

In [ ]:
# Open the files:
base_path = r"H:\Meu Drive\Py Projects\CVM 555\Fundos_555"
arquivo_zip = "inf_diario_fi_{}{}.zip"
arquivo_csv = "inf_diario_fi_{}{:02d}.csv"

# Specify the range of years and months you want to download
start_year = 2021
end_year = 2023
start_month = 1
end_month = 12

download_directory = "Fundos_555"

# Store DataFrames in a list for concatenation
dataframes_to_concat = []

for year in range(start_year, end_year + 1):
    for month in range(start_month, end_month + 1):
        archive_name = arquivo_zip.format(year, month)
        archive_path = os.path.join(base_path, archive_name)
        csv_name = arquivo_csv.format(year, month)

        with zipfile.ZipFile(archive_path) as zip_file:
            with zip_file.open(csv_name) as csv_file:
                lines = csv_file.readlines()

                # Clean and format data
                lines = [i.strip().decode('ISO-8859-1') for i in lines]
                lines = [i.split(';') for i in lines]

                # Create a DataFrame for the current CSV
                if_to_add = pd.DataFrame(lines[1:], columns=lines[0])
                dataframes_to_concat.append(if_to_add)

# Concatenate all DataFrames in the list
if_cvm_raw = pd.concat(dataframes_to_concat, ignore_index=True)

if_cvm_raw

In [ ]:
#transform the values on the column 'NR_COTST' to numeric and keep all the pandas dataframe
if_cvm_raw['NR_COTST'] = pd.to_numeric(if_cvm_raw['NR_COTST'], errors='coerce')
#filter to get only values on column 'nr_cotst' higher than 100
if_cvm_raw = if_cvm_raw[if_cvm_raw['NR_COTST'] > 100]
if_cvm_raw

In [ ]:
#download funds informations:
url = "https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv"
cadastro_raw = pd.read_csv(url,sep=';',encoding='ISO-8859-1')   

#choose only the columns that I want to see
cadastro_cvm = cadastro_raw[['TP_FUNDO','CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_REG', 'DT_CONST', 'CD_CVM', 'DT_CANCEL', 'SIT', 'CLASSE', 'DT_INI_CLASSE', 'FUNDO_EXCLUSIVO','TAXA_PERFM','INF_TAXA_PERFM', 'TAXA_ADM', 'INF_TAXA_ADM','DIRETOR', 'CNPJ_ADMIN', 'ADMIN','PF_PJ_GESTOR','CPF_CNPJ_GESTOR', 'GESTOR','CONTROLADOR', 'CLASSE_ANBIMA']]
cadastro_merge = cadastro_cvm[['TP_FUNDO','CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_REG', 'DT_CONST', 'CD_CVM', 'DT_CANCEL', 'SIT', 'CLASSE', 'FUNDO_EXCLUSIVO', 'GESTOR', 'CLASSE_ANBIMA']]

In [ ]:
#merge both DFs
if_cadastro_cvm = if_cvm_raw.merge(cadastro_merge,on='CNPJ_FUNDO', suffixes=('_if','_cd'))

In [ ]:
#filter to get only the funds with more than 100 investors. Exclude 'exclusive funds' just to make sure.
if_filter = if_cadastro_cvm[(if_cadastro_cvm['NR_COTST'] > 100) & (if_cadastro_cvm['FUNDO_EXCLUSIVO'] == "N")]

In [ ]:
#check values on fund classifications:
classe_anbima = if_filter[['CLASSE','CLASSE_ANBIMA']].drop_duplicates()
classe_anbima
#.csv file is still easier for me to read over
classe_anbima.to_csv('classe_anbima.csv', index=False)

In [ ]:
#Create DF to evaluate net money flows:
fundos_moneyflow = if_filter[['DT_COMPTC', 'CLASSE','CAPTC_DIA', 'RESG_DIA']].copy()

# Convert 'DT_COMPTC' column to datetime
fundos_moneyflow['DT_COMPTC'] = pd.to_datetime(fundos_moneyflow['DT_COMPTC'])

# Select columns 'CAPTC_DIA' and 'RESG_DIA' and convert to float
fundos_moneyflow[['CAPTC_DIA', 'RESG_DIA']] = fundos_moneyflow[['CAPTC_DIA', 'RESG_DIA']].astype(float)
fundos_moneyflow['CLASSE']=fundos_moneyflow['CLASSE'].astype(str)
# Create a new column 'NET_CAPTC'
fundos_moneyflow['NET_CAPTC'] = fundos_moneyflow['CAPTC_DIA'] - fundos_moneyflow['RESG_DIA']

# Display the resulting DataFrame and check formats
fundos_moneyflow = pd.DataFrame(fundos_moneyflow[fundos_moneyflow['CLASSE'].isin(['Fundo de Ações', 'Fundo de Renda Fixa', 'Fundo Multimercado'])])
fundos_moneyflow.info()

In [ ]:
#adjust to numeric and friendly format - hard time learning how to properly adjust it the way i want!
pd.options.display.float_format = '{:,.2f}'.format

fundos_group = fundos_moneyflow.groupby(['CLASSE','DT_COMPTC'])['NET_CAPTC'].sum()
#groupby the fundos_moneyflow DataFrame by 'CLASSE' and 'DT_COMPTC' and sum the 'NET_CAPTC' column
fundos_group = pd.DataFrame(fundos_group)
#add the cumulative sum of the 'NET_CAPTC' column
fundos_group['CUM_NET_CAPTC'] = fundos_group.groupby(level=0)['NET_CAPTC'].cumsum()
fundos_group
##plot a line chart of the 'CUM_NET_CAPTC' column for each 'CLASSE' and format the y-axis in millions BRL
fundos_group.unstack(level=0)['CUM_NET_CAPTC'].plot(figsize=(15,8), title='Captação Líquida por Classe de Fundo', ylabel='Captação Líquida')
#fix the y-axis tick labels
plt.gca().yaxis.set_major_formatter('{x:,.0f}')

#fix the x-axis tick labels and add a legend
plt.gca().set_xticklabels(fundos_group.index.get_level_values(1).strftime('%Y-%m'))
plt.legend(title='Classe de Fundo', loc='upper left')

#show graph for evaluating money flows for Fixed Income, Equity and Hedge Funds over the period
plt.show()